In [1]:
import pandas as pd
import sqlite3

In [5]:
df = pd.read_csv('tn.movie_budgets.csv.gz')
df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [11]:
df['release_date'] = pd.to_datetime(df['release_date'])

In [15]:
df['production_budget'] = df['production_budget'].str.strip('$')

In [31]:
df['production_budget'] = df['production_budget'].str.replace(',', '', regex=True).astype(float)

In [37]:
df['production_budget'] = df['production_budget'].astype(int)

In [39]:
df['domestic_gross'] = df['domestic_gross'].str.strip('$')

In [41]:
df['domestic_gross'] = df['domestic_gross'].str.replace(',', '', regex=True).astype(int)

In [43]:
df['worldwide_gross'] = df['worldwide_gross'].str.strip('$')

In [45]:
df['worldwide_gross'] = df['worldwide_gross'].str.replace(',', '', regex=True).astype(int)

In [110]:
df = df.drop(['total_rev'], axis = 1)

In [112]:
conn = sqlite3.connect('movie_budgets.db')

In [114]:
df.to_sql('movie_budgets_table', conn, if_exists='replace', index=False)

5782

The films with the highest returns are animations, drama, sequels, and sci-fi films.
They were made between 1997 and 2018.

In [116]:
query = '''
SELECT movie
FROM movie_budgets_table
ORDER BY worldwide_gross DESC
LIMIT 10

'''
result = pd.read_sql_query(query, conn)
print(result)

                                           movie
0                                         Avatar
1                                        Titanic
2           Star Wars Ep. VII: The Force Awakens
3                         Avengers: Infinity War
4                                 Jurassic World
5                                      Furious 7
6                                   The Avengers
7                        Avengers: Age of Ultron
8                                  Black Panther
9  Harry Potter and the Deathly Hallows: Part II


In [145]:
query = '''
SELECT movie
FROM movie_budgets_table
WHERE (domestic_gross >= 2*(production_budget))
LIMIT 10
'''
result = pd.read_sql_query(query, conn)
print(result)

                                  movie
0  Star Wars Ep. VII: The Force Awakens
1                Avengers: Infinity War
2                          The Avengers
3                        Jurassic World
4                         Black Panther
5                               Titanic
6                         Incredibles 2
7          Rogue One: A Star Wars Story
8                          Finding Dory
9                           Toy Story 3


A movie is generally considered a financial success if its ROI is at least 1, meaning it made 2x its budget.

The above movies that are financially successful are mostly sequels. 